# Association Rules Mining with MLxtend_library

Association analysis using the apriori algorithm and the MLxtend library. 

- [MLxtend documentation example](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) for generation of frequent item sets with Apriori algorithm
- [MLxtend documentation example](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/) for Association Rules


**Data Sources:**

- `data/raw/sales_total.csv`: real transaction dataset for a B2B retailer.

**Changes**

- 2019-07-07: Start notebook


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries,-load-data" data-toc-modified-id="Import-libraries,-load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import libraries, load data</a></span></li><li><span><a href="#Prepare-basic-data-structure" data-toc-modified-id="Prepare-basic-data-structure-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare basic data structure</a></span></li></ul></div>

---

## Import libraries, load data

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from tqdm import tqdm

# Specials
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

# My functions
import EDA_functions as EDA
import cleaning_functions as cleaning

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns 
# sns.set_style('whitegrid')
color = 'rebeccapurple'
%matplotlib inline

# Display settings
from IPython.display import display
pd.options.display.max_columns = 100

In [2]:
# Load data
transactions_raw = pd.read_csv('data/raw/sales_total.csv', parse_dates=['Fakturadatum'])

## Prepare basic data structure

**IMPORTANT:** Using MLxtnd this analysis requires that all the data for a transaction be included in 1 row and the items should be 1-hot encoded.

In [3]:
transactions_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2835054 entries, 0 to 2835053
Data columns (total 12 columns):
Kunde           int64
Fakturadatum    datetime64[ns]
Faktura         int64
Pos             int64
Artikel         object
Unit Price      float64
pro             int64
ME              object
Menge           float64
ME.1            object
Nettowert       float64
Währg           object
dtypes: datetime64[ns](1), float64(3), int64(4), object(4)
memory usage: 259.6+ MB


In [4]:
transactions_raw.sample(2)

,Kunde,Fakturadatum,Faktura,Pos,Artikel,Unit Price,pro,ME,Menge,ME.1,Nettowert,Währg
1892863,8154026,2018-05-15,92067512,5,4909801,44.85,100,ST,20.0,ST,8.97,CHF
1338613,7014163,2017-12-15,91896478,16,5186346,38.62,100,ST,50.0,ST,19.31,CHF


In [5]:
"""Clean data"""

# Look at 2018 data only
transactions_18_full = transactions_raw.loc[transactions_raw['Fakturadatum'].dt.year == 2018]
transactions_18_full = transactions_18_full[['Kunde', 'Fakturadatum', 'Artikel']]
 # Kick out all artikel that contain str values in their code
print("Unique artikel before cleaning:", transactions_18_full['Artikel'].nunique())
transactions_18_full['Artikel'] = pd.to_numeric(transactions_18_full['Artikel'], errors='coerce')
transactions_18 = transactions_18_full.dropna(how='any')
print("Unique artikel after cleaning:", transactions_18['Artikel'].nunique())
 # Kick-out special customers
transactions_18 = transactions_18.loc[transactions_18['Kunde'] > 700000]

transactions_18_grouped = pd.DataFrame(transactions_18.groupby(
        ['Kunde', 'Fakturadatum'])['Artikel'].unique())
transactions = transactions_18_grouped.reset_index(drop=True)

Unique artikel before cleaning: 74126
Unique artikel after cleaning: 49709


**Note:** By keeping numeric artikel IDs only I loose about a third of artikel!

In [7]:
# Check results
transactions.head()

,Artikel
0,"[9900179.0, 4100130.0, 5308748.0, 5074021.0, 5..."
1,"[2960232.0, 5308591.0, 6436018.0, 66933.0, 629..."
2,"[4921011.0, 2947803.0, 2550805.0, 2458003.0, 6..."
3,"[7947701.0, 6634351.0, 7906101.0, 8621604.0, 6..."
4,"[2119635.0, 2119634.0, 2310306.0, 6103861.0, 8..."


In [8]:
"""OHE to sparse format with MLxtnd TransactionEncoder"""

te = TransactionEncoder()
products_array = np.array(transactions['Artikel'])
products_array_ohe = te.fit(products_array).transform(products_array, sparse=True)
transactions_sparse = pd.SparseDataFrame(products_array_ohe, 
                                        columns=te.columns_, 
                                        default_fill_value=False)

assert transactions_sparse.iloc[1,].sum() == len(transactions.iloc[1,0])

In [9]:
# Check results
transactions_sparse.head()

,351.0,2809.0,2813.0,2814.0,6344.0,6731.0,8956.0,10067.0,10074.0,10075.0,10084.0,10128.0,10573.0,10921.0,13104.0,13105.0,13116.0,13117.0,13118.0,13121.0,13126.0,13127.0,13128.0,13129.0,13138.0,13140.0,13141.0,13142.0,13149.0,13150.0,13156.0,13157.0,13158.0,13223.0,13224.0,13225.0,13239.0,13243.0,13431.0,13432.0,13433.0,13932.0,13957.0,13961.0,13980.0,13984.0,16048.0,16173.0,16179.0,16190.0,...,629265325325.0,629265325375.0,629265325425.0,629265375325.0,629265375375.0,629265375425.0,629265375475.0,629265425375.0,629265425475.0,629265475325.0,629265475425.0,629265525325.0,629265575475.0,629265625325.0,629266275375.0,629266275425.0,629266325325.0,629266325375.0,629266325425.0,629266325475.0,629266375325.0,629266375375.0,629266375475.0,629266425425.0,629266425525.0,629266475475.0,629266525325.0,629266575325.0,629266575425.0,629266625325.0,629267275425.0,629267325325.0,629267325425.0,629267375325.0,629267375475.0,629267475375.0,629267475425.0,629267475475.0,629267525425.0,629267525525.0,629267575375.0,629267575425.0,629267725425.0,629275260360.0,629275460410.0,629275560410.0,629275710410.0,629277310310.0,629277310360.0,629277410410.0
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal